In [2]:
from game import Game2048
from game_env import Game2048Env

# Search content

In [4]:
# Definir modelo de NN por reforço
    # Métodos
        # Q-Learning
        # Monte Carlo
        # Gradiente de políticas
        # Aprendizado por diferença temporal
        # Trust Region Policy Optimization (TRPO)
# Definir parâmetros
    # Grid do jogo
# Definir penalidades
    # Tentar mover para uma direção sem movimentos válidos
    # Valores iguais/próximos muito dispersos? (viés?)
        # Aplicar penalidade apenas quando saiu do range (até 2 de distância?)
        # Verificar valores disponíveis no grid e contabilizar como valor próximo se estiver a até (3?) valores de distância
        # Não penalizar se valor for 2 ou 4.
# Definir recompensas
    # Juntar tiles
    # Manter valores da sequência em fileira ou próximos? (viés?)
    # 
# Definir estratégia de treino
    # Utilizar jogo já iniciado com mais tiles no grid? (aumentar aprendizado inicial sobre a regra do jogo)
        # Treinar o mesmo jogo com rotações diferentes

# Material:
# https://towardsdatascience.com/deep-q-learning-tutorial-mindqn-2a4c855abffc
# https://www.analyticsvidhya.com/blog/2019/04/introduction-deep-q-learning-python
# https://www.analyticsvidhya.com/blog/2019/01/monte-carlo-tree-search-introduction-algorithm-deepmind-alphago
# https://medium.com/@samina.amin/deep-q-learning-dqn-71c109586bae
# https://www.geeksforgeeks.org/implementing-deep-q-learning-using-tensorflow/

# Model

In [3]:
# pip install tensorflow==2.9.0 keras-rl2
# pip uninstall numpy -y
# pip install numpy==1.24.3

import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Activation
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

2025-03-15 14:07:20.110974: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-03-15 14:07:20.111015: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:

nActions = 4
tableSize = 4
hiddenFeatures = 16
temporalWindow = 5
env = Game2048Env()

model = Sequential()
model.add(Flatten(input_shape=(temporalWindow, tableSize, tableSize)))

model.add(Dense(hiddenFeatures))
model.add(Activation(activation="relu"))
# model.add(Activation(activation="leaky_relu")) # Tune alpha
# model.add(Activation(activation="elu")) # Tune alpha

model.add(Dense(nActions))
model.add(Activation(activation="linear"))

policy = EpsGreedyQPolicy()
replayMemory = SequentialMemory(limit=100, window_length=temporalWindow)
dqn = DQNAgent(
    model=model,
    nb_actions=nActions,
    memory=replayMemory,
    nb_steps_warmup=100,
    target_model_update=0.01,
    policy=policy
)
dqn.compile(optimizer=Adam(lr=0.001), metrics=["mae"])
dqn.fit(env=env, nb_steps=5000, visualize=False, verbose=2)


/usr/local/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)
2025-03-15 14:07:26.017082: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-03-15 14:07:26.017118: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2025-03-15 14:07:26.017135: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (623738dcea4e): /proc/driver/nvidia/version does not exist
2025-03-15 14:07:26.017360: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA


Training for 5000 steps ...


/usr/local/lib/python3.9/site-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  391/5000: episode: 1, duration: 1.628s, episode steps: 391, steps per second: 240, episode reward: 1500.000, mean reward:  3.836 [ 0.000, 128.000], mean action: 1.143 [0.000, 3.000],  loss: 73.696651, mae: 31.797598, mean_q: 54.681725
  806/5000: episode: 2, duration: 1.841s, episode steps: 415, steps per second: 225, episode reward: 1392.000, mean reward:  3.354 [ 0.000, 132.000], mean action: 1.482 [0.000, 3.000],  loss: 48.198509, mae: 60.740032, mean_q: 89.795158
  874/5000: episode: 3, duration: 0.307s, episode steps:  68, steps per second: 221, episode reward: 264.000, mean reward:  3.882 [ 0.000, 44.000], mean action: 1.971 [0.000, 3.000],  loss: 117.348938, mae: 71.557304, mean_q: 100.110153
 1049/5000: episode: 4, duration: 0.781s, episode steps: 175, steps per second: 224, episode reward: 568.000, mean reward:  3.246 [ 0.000, 68.000], mean action: 0.926 [0.000, 3.000],  loss: 49.945045, mae: 37.616653, mean_q: 59.433437
 1216/5000: episode: 5, duration: 0.741s, episode step

# Simulate Game

In [ ]:
# Play the game with predicts

actionMapping = {
    0: "up",
    1: "down",
    2: "left",
    3: "right"
}

games = [Game2048() for _ in range(1)]
for i, game in enumerate(games):
    print(f"Game {i + 1} initial state:")
    game.render()
    while (not game.game_over):
        observation = np.expand_dims(game.grid, axis=0)[0]
        predictedAction = dqn.forward(observation)
        
        print(f"\nPredicted Action: move {actionMapping[predictedAction]}")
        game.move(actionMapping[predictedAction])
        
        print(f"Game {i + 1} currrent state:")
        game.render()

        input("\nPress any key to continue...")

    print(f"Game {i + 1} is over.")



# Manual Game

In [ ]:
moves = ['up', 'left', 'right', 'down']
directions = ["w", "a", "d", "s"]

games = [Game2048() for _ in range(1)]
for i, game in enumerate(games):
    print(f"Game {i + 1} initial state:")
    game.render()
    while (not game.game_over):
        try: 
            currentMove = moves[directions.index(input("Next move (w, a, s , d): "))]
        except ValueError:
            print("Invalid move, try again.")
            continue

        game.move(currentMove)
        print(f"Game {i + 1} after move '{currentMove}':")
        game.render()

    print(f"Game {i + 1} is over.")


